In [3]:
import pandas as pd
import json 
import numpy as np

In [4]:
with open('489377937557040278.json') as f:
    content = json.load(f)

In [5]:
def _transpose_sales(year: int, k: str):
    def _to_numeric(x):
        if isinstance(x, str):
            try:
                return float(x.replace(',', ''))
            except ValueError:
                return np.nan

    _df = pd.DataFrame(content[k])
    _df = _df.loc[:, ~_df.apply(lambda x: (x == '-').all())]
    _df = _df.drop(columns=['XH', 'ZJE'])
    _df.set_index('PURCHASER_NAME', inplace=True)
    rename_col = {}
    for col in _df.columns:
        if col.startswith('YF_'):
            rename_col[col] = str(year) + '-' + col.replace('YF_', '')
    _df.rename(columns=rename_col, inplace=True)
    _df = _df.transpose()
    _df = _df.map(_to_numeric)
    _df.reset_index(inplace=True, drop=False)
    _df.columns.name = None
    _df.rename(columns={'index': 'month'}, inplace=True)
    _df['month'] = pd.to_datetime(_df['month'])
    _df.sort_values(by=['month'], inplace=True, ascending=False)
    _df.reset_index(inplace=True, drop=True)
    _df.drop(columns=['总计'], inplace=True)
    return _df


y1 = 2023
_df1 = _transpose_sales(y1, "customerDetail_24")
_df2 = _transpose_sales(y2 := y1 - 1, "customerDetail_25")
_df3 = _transpose_sales(y3 := y1 - 2, "customerDetail_26")
_df4 = _transpose_sales(y4 := y1 - 3, "customerDetail_27")
_dfconcat = pd.concat([_df1, _df2, _df3, _df4])
_dfconcat.replace(np.nan, None, inplace=True)
_dfconcat.sort_values(by='month', inplace=True)
print(_dfconcat)

data = []
months = _dfconcat['month'].apply(lambda x: x.strftime("%Y-%m")).tolist()
months.insert(0, '月份')
data.append(months)

for col in _dfconcat.columns[1:]:
    data.append([col] + _dfconcat[col].tolist())

content['customerRatioChart'] = data

# concentrationRateChart
sorted1 = _df1.drop(columns=['month']).sum().sort_values(ascending=False)
sorted2 = _df2.drop(columns=['month']).sum().sort_values(ascending=False)
sorted3 = _df3.drop(columns=['month']).sum().sort_values(ascending=False)
sorted4 = _df4.drop(columns=['month']).sum().sort_values(ascending=False)

cr1_s1 = sorted1[sorted1.index != '其他'].head(1).sum() / sorted1.sum()
cr5_s1 = sorted1[sorted1.index != '其他'].head(5).sum() / sorted1.sum()
cr10_s1 = sorted1[sorted1.index != '其他'].head(10).sum() / sorted1.sum()
cr20_s1 = sorted1[sorted1.index != '其他'].head(20).sum() / sorted1.sum()

cr1_s2 = sorted2[sorted2.index != '其他'].head(1).sum() / sorted2.sum()
cr5_s2 = sorted2[sorted2.index != '其他'].head(5).sum() / sorted2.sum()
cr10_s2 = sorted2[sorted2.index != '其他'].head(10).sum() / sorted2.sum()
cr20_s2 = sorted2[sorted2.index != '其他'].head(20).sum() / sorted2.sum()

cr1_s3 = sorted3[sorted3.index != '其他'].head(1).sum() / sorted3.sum()
cr5_s3 = sorted3[sorted3.index != '其他'].head(5).sum() / sorted3.sum()
cr10_s3 = sorted3[sorted3.index != '其他'].head(10).sum() / sorted3.sum()
cr20_s3 = sorted3[sorted3.index != '其他'].head(20).sum() / sorted3.sum()

cr1_s4 = sorted4[sorted4.index != '其他'].head(1).sum() / sorted4.sum()
cr5_s4 = sorted4[sorted4.index != '其他'].head(5).sum() / sorted4.sum()
cr10_s4 = sorted4[sorted4.index != '其他'].head(10).sum() / sorted4.sum()
cr20_s4 = sorted4[sorted4.index != '其他'].head(20).sum() / sorted4.sum()

x1_max_m, x1_min_m = _df1['month'].max().strftime("%m"), _df1['month'].min().strftime("%m")
x4_max_m, x4_min_m = _df4['month'].max().strftime("%m"), _df4['month'].min().strftime("%m")
x_1 = f"{y1}年{x1_min_m}月-{x1_min_m}月"
x_4 = f"{y4}年{x4_max_m}月-{x4_max_m}月"

x_axis = [x_1, f"{y2}年", f"{y3}年", x_4]
legend = ['CR1', 'CR5', 'CR10', 'CR20']

series = [
    [cr1_s1, cr1_s2, cr1_s3, cr1_s4],
    [cr5_s1, cr5_s2, cr5_s3, cr5_s4],
    [cr10_s1, cr10_s2, cr10_s3, cr10_s4],
    [cr20_s1, cr20_s2, cr20_s3, cr20_s4]
]
summary_cr = (f"{x_axis[0]}第一大客户销售额占比(CR1){cr1_s1:.2%}，同比变化{cr1_s1 / cr1_s2 - 1:.2%}"
              f"前五大客户销售额占比(CR5){cr5_s1:.2%}，同比变化{cr5_s1 / cr1_s2 - 1:.2%}"
              f"前10大客户占比(CR10){cr10_s1:.2%}，同比变化{cr10_s1 / cr10_s2 - 1:.2%}"
              f"前20大客户占比(CR20){cr20_s1:.2%}, 同比变化{cr20_s1 / cr20_s2 - 1:.2%}。")

summary_cr += (f"{x_axis[1]}第一大客户销售额占比(CR1){cr1_s2:.2%}，同比变化{cr1_s2 / cr1_s3 - 1:.2%}"
               f"前五大客户销售额占比(CR5){cr5_s2:.2%}，同比变化{cr5_s2 / cr1_s3 - 1:.2%}"
               f"前10大客户占比(CR10){cr10_s2:.2%}，同比变化{cr10_s2 / cr10_s3 - 1:.2%}"
               f"前20大客户占比(CR20){cr20_s2:.2%}, 同比变化{cr20_s2 / cr20_s3 - 1:.2%}。")

comment = "前五大客户的销售额占比（CR5）是衡量市场集中度的重要指标。其可参考如下分级：\n0-25%显示市场高度分散；25%-40%表明客户分布较广；40%-50%反映市场均衡；50-70%指市场较集中；而70%-100%则意味着市场高度集中。"

content['concentrationRateChart'] = {
    'x_axis': x_axis,
    'series': series,
    'legend': legend,
    'summary': summary_cr,
    'comment': comment
}

# customerSalesChart
sales = []
for col in _dfconcat.columns[1:]:
    sales.append(_dfconcat[col].tolist())
companies = _dfconcat.columns[1:].tolist()
content['customerSalesChart'] = {
    'series-list': sales,
    'legend': companies,
    'x-axis': months[1:]

}

        month  深圳禾苗通信科技有限公司 湖南博深供应链有限公司 仁寿县卓航电子科技有限公司 湖南中芯供应链有限公司 宜宾创益科技有限公司  \
1  2020-11-01       4399.77        None          None        None       None   
0  2020-12-01      13005.74        None          None        None       None   
11 2021-01-01       8700.76        None          None         0.0       None   
10 2021-02-01       4638.06        None          None         0.0       None   
9  2021-03-01       8154.34        None          None         0.0       None   
8  2021-04-01       7444.75        None          None         0.0       None   
7  2021-05-01       6897.25        None          None         0.0       None   
6  2021-06-01       5476.91        None          None         0.0       None   
5  2021-07-01       6859.90        None          None         0.0       None   
4  2021-08-01       3361.94        None          None         0.0       None   
3  2021-09-01       6522.19        None          None         0.0       None   
2  2021-10-01       4778.99        None 

In [6]:
_dfconcat.reset_index(inplace=True, drop=True)
_dfconcat.replace(0, None, inplace=True)

In [7]:
fp_y = {}
lp_y = {}
for col in _dfconcat.drop(columns=['其他']).columns[1:]:
    # _dfconcat[col] = _dfconcat[col].astype(float)
    idx_valid = _dfconcat[col].first_valid_index()
    idx_valid_last = _dfconcat[col].last_valid_index()
    t = _dfconcat.loc[idx_valid, 'month']
    tl = _dfconcat.loc[idx_valid_last, 'month']
    fp_y[col] = t.year
    lp_y[col] = tl.year

In [8]:
_dfconcat['year'] = _dfconcat['month'].apply(lambda x: x.year)
_dfconcat

,month,深圳禾苗通信科技有限公司,湖南博深供应链有限公司,仁寿县卓航电子科技有限公司,湖南中芯供应链有限公司,宜宾创益科技有限公司,泸州禾苗创先智能科技有限公司,深圳市至友时代科技有限公司,深圳市朗华供应链服务有限公司,研祥智能科技股份有限公司,...,深圳市伟安特电子有限公司,深圳市诚旗电子有限公司,深圳市大道同维科技有限公司,深圳前海蚂蚁芯城科技有限公司,深圳百锐科技有限公司,上海锐伟电子科技有限公司,深圳鼎鸿创展科技有限公司,深圳市高信技术有限公司,深圳市乐众云科技有限公司,year
0,2020-11-01,4399.77,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2020
1,2020-12-01,13005.74,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,34.36,2020
2,2021-01-01,8700.76,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2021
3,2021-02-01,4638.06,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2021
4,2021-03-01,8154.34,None,None,None,None,None,None,None,None,...,28.88,None,20.35,16.51,None,None,None,None,None,2021
5,2021-04-01,7444.75,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2021
6,2021-05-01,6897.25,None,None,None,None,None,None,None,None,...,None,None,4.55,None,None,None,8.25,None,None,2021
7,2021-06-01,5476.91,None,None,None,None,None,None,None,None,...,None,27.53,None,None,None,None,None,None,None,2021
8,2021-07-01,6859.90,None,None,None,None,None,None,None,None,...,None,None,None,None,11.83,None,None,None,None,2021
9,2021-08-01,3361.94,None,None,None,None,None,None,None,None,...,None,None,None,0.75,None,None,None,None,None,2021


In [9]:
df_temp = _dfconcat.drop(columns=['month', '其他'])
res = df_temp.groupby('year').sum()
stat = []
for i in range(1, len(res)):
    curr, prev = 0, 0
    curr_count, prev_count = 0, 0
    y = res.index[i]
    for com in res.iloc[i].index:
        # print(com)
        y_f = fp_y.get(com)
        if y_f >= y:
            # 新客户
            curr += res.iloc[i][com]
            curr_count += 1
        else:
            prev += res.iloc[i][com]
            prev_count += 1
    # 
    stat.append({
        'new_customer': curr,
        'repeat_customer': prev,
        'year': y, 
        'new_customer_count': curr_count,
        'repeat_customer_count': prev_count
    })
        
            
stat
         

[{'new_customer': 2519.5400000000004,
  'repeat_customer': 79532.28,
  'year': 2021,
  'new_customer_count': 48,
  'repeat_customer_count': 3},
 {'new_customer': 15874.150000000001,
  'repeat_customer': 95748.0,
  'year': 2022,
  'new_customer_count': 30,
  'repeat_customer_count': 21},
 {'new_customer': 32749.619999999995,
  'repeat_customer': 170293.91999999998,
  'year': 2023,
  'new_customer_count': 14,
  'repeat_customer_count': 37}]

In [10]:
df_stat = pd.DataFrame(stat)
df_stat['repeat_customer_ratio'] = df_stat['repeat_customer'] / (df_stat['new_customer'] + df_stat['repeat_customer'])
df_stat['new_customer_ratio'] = 1 - df_stat['repeat_customer_ratio']
df_stat.sort_values(by='year', ascending=False)

,new_customer,repeat_customer,year,new_customer_count,repeat_customer_count,repeat_customer_ratio,new_customer_ratio
2,32749.62,170293.92,2023,14,37,0.838706,0.161294
1,15874.15,95748.00,2022,30,21,0.857787,0.142213
0,2519.54,79532.28,2021,48,3,0.969293,0.030707


In [11]:
res

,深圳禾苗通信科技有限公司,湖南博深供应链有限公司,仁寿县卓航电子科技有限公司,湖南中芯供应链有限公司,宜宾创益科技有限公司,泸州禾苗创先智能科技有限公司,深圳市至友时代科技有限公司,深圳市朗华供应链服务有限公司,研祥智能科技股份有限公司,福州领派兄弟科技有限公司,...,广东九联科技股份有限公司,深圳市伟安特电子有限公司,深圳市诚旗电子有限公司,深圳市大道同维科技有限公司,深圳前海蚂蚁芯城科技有限公司,深圳百锐科技有限公司,上海锐伟电子科技有限公司,深圳鼎鸿创展科技有限公司,深圳市高信技术有限公司,深圳市乐众云科技有限公司
year,,,,,,,,,,,,,,,,,,,,,
2020,17405.51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,34.36
2021,79516.67,0,0,518.61,0,0,0,0,0,0,...,41.04,28.88,27.53,24.9,22.19,11.83,11.06,8.65,1.67,0
2022,89476.49,6474.18,0,6031.37,0,0,960.02,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023,116813.23,40931.19,24193.41,11460.29,4127.44,2040.89,980.78,771.89,735.18,362.79,...,0,0,0,0,0,0,0,0,0,0


In [12]:
s1 = pd.Series(fp_y)
s2 = pd.Series(lp_y)

In [13]:
# concat s1 s2, with given col name a1, a2
# d1 = pd.concat([s1, s2], axis=1)
d1 = pd.concat([s1, s2], axis=1, keys=['first', 'last'])
d1

,first,last
深圳禾苗通信科技有限公司,2020,2023
湖南博深供应链有限公司,2022,2023
仁寿县卓航电子科技有限公司,2023,2023
湖南中芯供应链有限公司,2021,2023
宜宾创益科技有限公司,2023,2023
泸州禾苗创先智能科技有限公司,2023,2023
深圳市至友时代科技有限公司,2022,2023
深圳市朗华供应链服务有限公司,2023,2023
研祥智能科技股份有限公司,2023,2023
福州领派兄弟科技有限公司,2023,2023


In [14]:
df_cr = d1[(d1['first'] <2023) & (d1['last'] >= 2023)]
df_cr

,first,last
深圳禾苗通信科技有限公司,2020,2023
湖南博深供应链有限公司,2022,2023
湖南中芯供应链有限公司,2021,2023
深圳市至友时代科技有限公司,2022,2023
锐飞（深圳）智慧科技有限公司,2022,2023
深圳市华宇扬贸易有限公司,2022,2023


In [15]:
dcurr = res[df_cr.index.tolist()].loc[2023]
dcurr

深圳禾苗通信科技有限公司      116813.23
湖南博深供应链有限公司        40931.19
湖南中芯供应链有限公司        11460.29
深圳市至友时代科技有限公司        980.78
锐飞（深圳）智慧科技有限公司        73.59
深圳市华宇扬贸易有限公司          34.84
Name: 2023, dtype: object

In [16]:
r1 = dcurr / res.loc[2023].sum()
r2: pd.Series = dcurr / res.loc[2022].sum()

In [17]:
dfunion = pd.concat([r1, r2], axis=1, keys=['1','2'])
dfunion

,1,2
深圳禾苗通信科技有限公司,0.575311,1.046506
湖南博深供应链有限公司,0.201588,0.366694
湖南中芯供应链有限公司,0.056443,0.10267
深圳市至友时代科技有限公司,0.00483,0.008787
锐飞（深圳）智慧科技有限公司,0.000362,0.000659
深圳市华宇扬贸易有限公司,0.000172,0.000312


In [18]:
[{k: v} for k, v in r2.to_dict().items()]


[{'深圳禾苗通信科技有限公司': 1.0465058234409568},
 {'湖南博深供应链有限公司': 0.3666941552371102},
 {'湖南中芯供应链有限公司': 0.10267039292828528},
 {'深圳市至友时代科技有限公司': 0.00878660731763364},
 {'锐飞（深圳）智慧科技有限公司': 0.0006592777508765062},
 {'深圳市华宇扬贸易有限公司': 0.0003121244305005772}]

In [19]:
r1

深圳禾苗通信科技有限公司      0.575311
湖南博深供应链有限公司       0.201588
湖南中芯供应链有限公司       0.056443
深圳市至友时代科技有限公司      0.00483
锐飞（深圳）智慧科技有限公司    0.000362
深圳市华宇扬贸易有限公司      0.000172
Name: 2023, dtype: object

In [28]:
r1.

array([0.575311236200866, 0.20158824063055636, 0.056442524593493575,
       0.004830392535512333, 0.0003624345793025476,
       0.00017158881292160291], dtype=object)

In [1]:
a = [1,2,3]

In [4]:
a.reverse()

In [5]:
a

[3, 2, 1]

In [6]:
a[:2]

[3, 2]

In [7]:
a.copy()

[3, 2, 1]

In [1]:
import pandas as pd

In [23]:
s = pd.Series([100,49])

In [24]:
s

0    100
1     49
dtype: int64

In [26]:
round(s / s.shift(periods=-1) - 1, 1) 


0    1.0
1    NaN
dtype: float64

In [9]:
s.shift(periods=-1)

0    50.0
1     NaN
dtype: float64

In [13]:
a = [1,2,3]

In [18]:
a[:2]

[1, 2]

In [19]:
s.round()

0    100
1     50
dtype: int64